A file with columns for each repeat would be great thanks. I’ve attached the data in this email. I’m basing the species composition of the core in your biogeosciences paper, so I’m trying to construct a regression line with the following weighting, G. oceanica 45%, E. huxleyi 35%, C. pelagicus 15%, C. Leptoporus 5%.
 
Annoyingly, I have the fewest observations for G. Oceanica so this potentially wont work given that this will end up being a sample of 1 for C. Leptoporus? I think a regression line with equal weighting and a discussion of how this will lead to error in cell size estimates is the nest best option.
 
Great, thanks for having a look at the manuscript. End of the week should be do-able!
 
 

### Import required modules

In [24]:
import pandas as pd
import numpy as np

### Read in the data from the excel file

In [25]:
file = '/Users/ph290/Downloads/Diameter_cell.xlsx'
diameter_data = pd.read_excel(file,sheet_name = None)

### Set the no. individuals to use for teh subsampling
here it is set to the minimum nuber in a any of the species' datasets

In [26]:
min_no_samples = np.min([len(diameter_data[species]['Cell']) for species in list(diameter_data)])

### Loop through and create the subsets

In [172]:
no_subsets = 100

df = pd.DataFrame(columns = list(diameter_data))
mydict={}

for species in list(diameter_data):
    mydict[species] = {}
    for i in np.arange(no_subsets):
        no_items = len(diameter_data[species]['Cell'])
        index_values = np.random.choice(no_items, min_no_samples, replace = False)
        mydict[species][str(i)] = {}
        mydict[species][str(i)]['Coccosphere'] = diameter_data[species]['Coccosphere'].iloc[index_values].values
        mydict[species][str(i)]['Cell'] = diameter_data[species]['Cell'].iloc[index_values].values


In [173]:
columns = []

for species in list(mydict):
    for i in np.arange(no_subsets):
        for j in ['Coccosphere','Cell']:
            columns.append(species+'_'+j+'_'+str(i))

data = np.zeros([min_no_samples,len(columns)])
data[:] = np.nan

In [174]:
for y in range(min_no_samples):
    x = 0
    for species in list(mydict):
        for i in np.arange(no_subsets):
            for j in ['Coccosphere','Cell']:
                data[y,x] = mydict[species][str(i)][j][y]
                x += 1

In [175]:
df = pd.DataFrame (data, columns = columns)

In [176]:
df.to_csv('/Users/ph290/Downloads/Diameter_cell_subsets.csv', index=False)  